In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv("./Final_Data.csv")

In [2]:
data.head()

,Unnamed: 0,id,brand,title,sold_price,actual_price,url
0,0,1,CHALODIA,Women Flared High Rise Black Jeans,₹549,"₹1,999",https://www.flipkart.com/chalodia-flared-women...
1,1,2,BLINKIN,Color Block Women Green Tights,₹349,"₹1,680",https://www.flipkart.com/blinkin-color-block-w...
2,2,3,Yana,Pack of 3 Women Sports Non Padded Bra,₹149,₹890,https://www.flipkart.com/yana-women-sports-non...
3,3,4,KOTTY,Women Regular Fit Black Viscose Rayon Trousers,₹349,"₹1,999",https://www.flipkart.com/kotty-regular-fit-wom...
4,4,5,TYFFYN,Women Slim High Rise Black Jeans,₹499,"₹1,499",https://www.flipkart.com/tyffyn-slim-women-bla...


In [8]:
item_dict = pd.Series(data.url.values,index=data.title).to_dict()

In [9]:
from sentence_transformers import SentenceTransformer
sentences =list(data.url.values)

model = SentenceTransformer('sentence-transformers/stsb-roberta-base')
embeddings = model.encode(sentences)
print(embeddings.shape)

(8945, 768)


In [34]:
np.save("embeddings.npy", embeddings)
np.save("items_urls.npy", data.url.values)

In [26]:
import faiss
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

distances,indices = index.search(embeddings[0].reshape((1, 768)), 5)

In [27]:
distances

array([[  0.      ,  30.533703,  58.272667,  79.90371 , 124.33252 ]],
      dtype=float32)

In [30]:
def returnTopNSimilar(query_string, n=5):
    embeddings = np.load("./embeddings.npy")
    item_urls = np.load("./items_urls.npy")
    from sentence_transformers import SentenceTransformer
    model = SentenceTransformer('sentence-transformers/stsb-roberta-base')
    query_vector = model.encode([query_string])
    
    distances,indices = index.search(query_vector, 5)
    
    return item_urls[indices]

In [31]:
returnTopNSimilar("Shoes black")

array([['https://www.flipkart.com/scube-designs-round-neck-women-blouse/p/itm86a878c330a54?pid=BLOGFJ59MGD9ZUZN&lid=LSTBLOGFJ59MGD9ZUZNLZJQLZ&marketplace=FLIPKART&store=clo%2F8on%2F5n9%2Fhny&srno=b_3_97&otracker=browse&fm=organic&iid=en_QdS%2BQkPoRK258iMgKrd0w2hcwLrvd%2FWckjKSkNCov6uhbSiaZuLPIvE%2F4AFywIb8kQB3LLuVcaWc1vmzi9USfw%3D%3D&ppt=None&ppn=None&ssid=4qso4sexlc0000001684490473167',
        'https://www.flipkart.com/w-liberty-women-black-bellies/p/itmb645f2554efae?pid=SNDG5ZQCXDEXMHA8&lid=LSTSNDG5ZQCXDEXMHA8J8H4HA&marketplace=FLIPKART&store=osp%2Fiko%2F974&srno=b_2_77&otracker=nmenu_sub_Women_0_Ballerinas&fm=organic&iid=en_c68HHChX4gjmM%2FDXZo2rMp0CPRP%2Fy2E7nYsLjGgJeaqtnQCBttV8UfFA6CkDEzEk%2FGQtL4e8jrde5laTZnQfzA%3D%3D&ppt=None&ppn=None&ssid=hl1opj6xxc0000001684490196900',
        'https://www.flipkart.com/levi-s-men-mid-calf-crew/p/itm3e8811ab17060?pid=SOCGEQYYDAWDXZZ3&lid=LSTSOCGEQYYDAWDXZZ3M3LZHU&marketplace=FLIPKART&store=clo%2Fqd8&srno=b_1_20&otracker=browse&fm=organic&iid=1